In [11]:
import osmnx as ox
import pandas as pd

In [14]:
# Example: get driving network for a city
G = ox.graph_from_place(
    "Somerville, Massachusetts, USA",
    retain_all=True,
    network_type="bike")

In [16]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [17]:
edges.sample(10)

,,,osmid,highway,lanes,maxspeed,name,width,oneway,reversed,length,geometry,ref,service,access,bridge,tunnel,junction
u,v,key,,,,,,,,,,,,,,,,
61171385,61171375,0,1432163179,tertiary,2,NaN,Curtis Street,12.2,False,False,16.724852,"LINESTRING (-71.12492 42.40685, -71.12482 42.4...",NaN,NaN,NaN,NaN,NaN,NaN
71942069,71929021,0,9429674,tertiary,2,25 mph,Temple Street,NaN,False,False,28.845697,"LINESTRING (-71.09153 42.39507, -71.09148 42.3...",NaN,NaN,NaN,NaN,NaN,NaN
7305257106,12347356457,0,782320823,service,NaN,NaN,NaN,NaN,False,False,2.674682,"LINESTRING (-71.10359 42.3882, -71.10356 42.38...",NaN,driveway,customers,NaN,NaN,NaN
61170988,71915250,0,9429708,residential,1,25 mph,Village Street,7.9,True,False,21.332367,"LINESTRING (-71.1046 42.38042, -71.10468 42.38...",NaN,NaN,NaN,NaN,NaN,NaN
71927154,71949282,0,9429821,residential,2,NaN,Kingston Street,12.2,False,False,25.036101,"LINESTRING (-71.12683 42.39712, -71.127 42.397...",NaN,NaN,NaN,NaN,NaN,NaN
61177298,11395248465,0,1228763126,residential,NaN,25 mph,Central Road,NaN,False,False,25.312155,"LINESTRING (-71.10212 42.3891, -71.10199 42.3893)",NaN,NaN,NaN,NaN,NaN,NaN
4261241460,61171493,0,8605311,secondary,2,25 mph,Broadway,24.4,True,False,24.615498,"LINESTRING (-71.09154 42.39142, -71.09133 42.3...",NaN,NaN,NaN,NaN,NaN,NaN
7490975006,71947551,0,9429591,residential,1,20 mph,Richardson Street,10.7,False,True,49.250636,"LINESTRING (-71.10492 42.39559, -71.10446 42.3...",NaN,NaN,NaN,NaN,NaN,NaN
5458857454,5458857445,0,1322009158,service,NaN,NaN,NaN,NaN,False,False,17.071976,"LINESTRING (-71.08452 42.37402, -71.08435 42.3...",NaN,driveway,NaN,NaN,NaN,NaN


In [14]:
highways = edges.reset_index()['highway']

In [24]:
unique = set()
for _, v in highways.items():
    if isinstance(v, list):
        unique.update(v)
    else:
        unique.add(v)
unique

{'busway',
 'cycleway',
 'living_street',
 'path',
 'pedestrian',
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'trunk',
 'trunk_link',
 'unclassified'}

In [ ]:
import numpy as np
import re

In [4]:
edges[['highway', 'name', 'maxspeed']].sample(10)

,,,highway,name,maxspeed
u,v,key,,,
7983674950,71916827,0,residential,Belmont Street,20 mph
11777326029,11777326029,1,service,NaN,NaN
61172626,61174071,0,trunk,Fellsway,NaN
1949661741,1155703191,0,residential,Latin Way,NaN
8334781996,8334781999,0,service,NaN,NaN
61170625,61170638,0,secondary,Summer Street,20 mph
61170914,71917815,0,secondary,Summer Street,20 mph
71914380,71927580,0,residential,Edgar Avenue,20 mph
71947029,71921397,0,secondary,Grand Union Boulevard,20 mph


In [7]:
def extract_maxspeed(x):
    # Handle NaN early
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan

    # Handle lists and numpy arrays uniformly
    if isinstance(x, (list, np.ndarray)):
        speeds = [extract_maxspeed(i) for i in x]
        # filter out NaNs
        speeds = [s for s in speeds if not pd.isna(s)]
        return max(speeds) if speeds else np.nan

    # Convert to string and extract digits
    x_str = str(x)
    nums = re.findall(r'\d+', x_str)

    return int(max(nums, key=int)) if nums else np.nan

In [8]:
edges['maxspeed_int'] = edges['maxspeed'].apply(extract_maxspeed)

In [9]:
edges[['highway', 'name', 'maxspeed', 'maxspeed_int']].sample(10)

,,,highway,name,maxspeed,maxspeed_int
u,v,key,,,,
71951018,71928807,0,residential,Stickney Avenue,20 mph,20.0
61173051,71938041,0,residential,Dickson Street,NaN,NaN
71916774,71953040,0,residential,Victoria Street,NaN,NaN
71954205,945798791,0,primary,Somerville Avenue,25 mph,25.0
61182360,61178893,0,secondary,Medford Street,25 mph,25.0
61177298,11395248465,0,residential,Central Road,25 mph,25.0
604063583,71936370,0,residential,Jaques Street,20 mph,20.0
71920826,71947231,0,secondary,Broadway,25 mph,25.0
71919864,71949322,0,residential,Century Street,20 mph,20.0


In [11]:
edges.to_file("../data/somerville_roads.gpkg", layer="roads", driver="GPKG")

## Bike

In [ ]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")

In [ ]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

edges_drive.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")
edges_bike.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")